In [ ]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

In [2]:
from bkmonitor.models.metric_list_cache import MetricListCache

groups = [
    # {"data_source_label", "result_table_id"},
    # {"data_source_label", "result_table_id", "bk_biz_id"},
    {"data_source_label", "result_table_id", "bk_biz_id", "data_type_label"},
]
# 用于保存需要查询的参数
param_dics_list = []

for item in groups:
    result = MetricListCache.objects.all().values(*item).distinct()[0:10000]
    param_dics_list.append(list(result))

In [3]:
def query_metric_cache_loop(pre_field_set):
    result = []
    for param_dics in param_dics_list:
        for metric_info in param_dics:
            query_set = MetricListCache.objects.filter(**metric_info).order_by("id").values(*pre_field_set)
            for item in query_set:
                result.append(item)
    result.sort()
    return result

# query_metric_cache_loop()




In [4]:
from functools import reduce
from itertools import count
from box import Box
from pandas import DataFrame
from collections import defaultdict


def query_metric_cache_batch(pre_field_set):
    # 过滤条件字典
    filter_conditions = defaultdict(list)
    metric_info_recommends = []

    field_set = pre_field_set.copy()

    for param_dics in param_dics_list:
        for metric_info in param_dics:
            for key, value in metric_info.items():
                filter_conditions[key + "__in"].append(value)

            metric_info_recommends.append(metric_info)
            # 更新查询需要显示的字段集合
            field_set.update(set(metric_info.keys()))

    # 批量获取所有需要查询的指标
    metric_data = MetricListCache.objects.filter(**filter_conditions).order_by("id").values(*field_set)

    df = DataFrame(metric_data)

    result = []

    def generate_conditions(x, y):
        """
        用于构建符合DataFrame所要求格式的过滤条件
        """
        if not isinstance(x, tuple):
            return x & (df[str(y[0])] == y[1])
        return (df[str(x[0])] == x[1]) & (df[str(y[0])] == y[1])

    for metric_info in metric_info_recommends:
        condition = reduce(generate_conditions, metric_info.items())
        metric = df[condition].to_dict()
        if not metric.get("id"):
            # 没有获取到目标数据，跳过
            continue

        # 获取到一个随机的key
        random_key = list(metric.keys())[0]
        # 事先创建好一个元素为空字典的列表
        current_res = [{} for _ in range(len(metric.get(random_key)))]
        index = count()
        for key, value_dic in metric.items():
            for item in value_dic.values():
                current_res[next(index)][key] = item
            index = count()
        result.extend(current_res)

    result.sort()
    return result

# query_metric_cache_batch(pre_field_set)


In [ ]:
# 测试使用pandas 进行过滤的数据与实际情况是否一致

pre_field_set = {"id",
                 "result_table_id",
                 "data_source_label",
                 "metric_field",
                 }

if __name__ == '__main__':
    # 循环获取并过滤数据    
    result_loop = query_metric_cache_loop(pre_field_set)
    # 只保存id
    result_loop = list(map(lambda x: x["id"], result_loop))
    # 批量获取数据，并使用pandas 进行过滤
    result_batch = query_metric_cache_batch(pre_field_set)
    result_batch = list(map(lambda x: x["id"], result_batch))
    print("量查询结果是否一致：", result_loop == result_batch)
    print(result_loop[0:10], len(result_loop))
    print()
    print(result_batch[0:10], len(result_batch))


In [ ]:
# 断点测试generate_recommended_metric_panels 方法
from fta_web.alert.resources import MetricRecommendationResource
from bkmonitor.aiops.alert.utils import RecommendMetricManager


params = [
    {'data_source_label': 'bk_monitor', 'result_table_id': 'strategy', 'data_type_label': 'alert', 'bk_biz_id': 17},
    {'data_source_label': 'bk_monitor', 'result_table_id': 'pingserver.base', 'data_type_label': 'time_series',
     'bk_biz_id': 0}
]
params = iter(params)

RecommendMetricManager(1, True).generate_recommended_metric_panels(1, {}, {})